In [43]:
import pandas as pd
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playoffpicture

# Get a list of team info
team_info = teams.get_teams()
head_coaches_data = []

# Iterate through teams and seasons
for team in team_info:
    team_id = team['id']
    team_name = team['abbreviation']
    for season in range(2005, 2024):
        coach_data = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
        coach_data_df = coach_data.coaches.get_data_frame()
        seasons = f"{season}-{str(season+1)[-2:]}"
        if not coach_data_df.empty:
            try:
                coach_name = coach_data_df[coach_data_df['COACH_TYPE'] == 'Head Coach']['COACH_NAME'].values[0]
            except IndexError:
                coach_name = 'None'
        head_coaches_data.append({
            'Team ID': team_id,
            'Season': seasons,
            'Team': team_name,
            'Coach': coach_name
        })

head_coaches_df = pd.DataFrame(head_coaches_data)

In [41]:
no_data = head_coaches_df[head_coaches_df['Coach'] == 'None']
no_data

,Team ID,Season,Team,Coach
66,1610612740,2014-15,NOP,None
121,1610612743,2012-13,DEN,None
200,1610612747,2015-16,LAL,None
349,1610612755,2012-13,PHI,None
374,1610612756,2018-19,PHX,None
409,1610612758,2015-16,SAC,None
446,1610612760,2014-15,OKC,None
567,1610612766,2021-22,CHA,None


In [23]:
import pandas as pd
from nba_api.stats.endpoints import teamyearbyyearstats
from nba_api.stats.static import teams

# Get a list of team info
team_info = teams.get_teams()
team_record_data_list = []  # List to store individual data frames

# Iterate through teams and seasons
for team in team_info:
    team_id = team['id']
    team_record_data = teamyearbyyearstats.TeamYearByYearStats(team_id=team_id)
    team_record_data_df = team_record_data.team_stats.get_data_frame()
    team_df = pd.DataFrame({
        'Team ID': team_id,
        'Season': team_record_data_df['YEAR'],
        'Team Name': team_record_data_df['TEAM_NAME'],
        'Wins': team_record_data_df['WINS'],
        'Losses': team_record_data_df['LOSSES'],
        'Win PCT': team_record_data_df['WIN_PCT']
    })
    team_record_data_list.append(team_df)

# Concatenate all data frames into the final result
team_record_df = pd.concat(team_record_data_list)


/var/folders/3z/8j8g_lgs5j7b31l1sp39n6380000gn/T/ipykernel_5140/2362937292.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_record_df = pd.concat([team_record_df, team_df])


,Team ID,Season,Team Name,Wins,Losses,Win PCT
0,1610612737,1949-50,Blackhawks,29,35,0.453
1,1610612737,1950-51,Blackhawks,25,43,0.368
2,1610612737,1951-52,Hawks,17,49,0.258
3,1610612737,1952-53,Hawks,27,44,0.380
4,1610612737,1953-54,Hawks,21,51,0.292
...,...,...,...,...,...,...
29,1610612766,2019-20,Hornets,23,42,0.354
30,1610612766,2020-21,Hornets,33,39,0.458
31,1610612766,2021-22,Hornets,43,39,0.524
32,1610612766,2022-23,Hornets,27,55,0.329


In [26]:
combined_df = pd.merge(head_coaches_df, team_record_df, on=['Team ID', 'Season'], how='left')
combined_df

,Season,Team ID,Team,Coach,Team Name,Wins,Losses,Win PCT
0,2005-06,1610612737,ATL,Mike Woodson,Hawks,26,56,0.317
1,2006-07,1610612737,ATL,Mike Woodson,Hawks,30,52,0.366
2,2007-08,1610612737,ATL,Mike Woodson,Hawks,37,45,0.451
3,2008-09,1610612737,ATL,Mike Woodson,Hawks,47,35,0.573
4,2009-10,1610612737,ATL,Mike Woodson,Hawks,53,29,0.646
...,...,...,...,...,...,...,...,...
535,2018-19,1610612766,CHA,James Borrego,Hornets,39,43,0.476
536,2019-20,1610612766,CHA,James Borrego,Hornets,23,42,0.354
537,2020-21,1610612766,CHA,James Borrego,Hornets,33,39,0.458
538,2021-22,1610612766,CHA,No Data,Hornets,43,39,0.524


In [76]:
from nba_api.stats.endpoints import playoffpicture
from nba_api.stats.static import teams
data = []
teams = teams.get_teams()

for team in teams: 
    for season in range(2000, 2024):
        playoff_picture = playoffpicture.PlayoffPicture(season_id='2'+str(season))
        playoff_picture_east_df = playoff_picture.east_conf_standings.get_data_frame()
        # playoff_picture_west_df = playoff_picture.west_conf_standings.get_data_frame()
        east_team_wins = playoff_picture_east_df['WINS']
        east_team_losses = playoff_picture_east_df['LOSSES']
        east_team_pct = playoff_picture_east_df['PCT']
        east_team = playoff_picture_east_df['TEAM']
        east_team_clinched_playoffs = playoff_picture_east_df['CLINCHED_PLAYOFFS']
        data.append({
            'Season': season, 
            'Team': team,
            'East Wins': east_team_wins,
            'East Losses': east_team_losses, 
            'East PCT': east_team_pct,
            'East Clinched': east_team_clinched_playoffs})

records = pd.DataFrame(data)s
records


# # Create an instance of the PlayoffPicture object
# playoff_picture = playoffpicture.PlayoffPicture(season_id='22005')

# # Call the get_data_frames method to retrieve the data frames
# data_frames = playoff_picture.east_conf_standings.get_data_frame()
# data_frames

,Season,Team,East Wins,East Losses,East PCT,East Clinched
0,2000,"{'id': 1610612737, 'full_name': 'Atlanta Hawks...",0 56 1 52 2 50 3 48 4 47 5...,0 26 1 30 2 32 3 34 4 35 5...,0 0.683 1 0.634 2 0.610 3 0.58...,0 None 1 None 2 None 3 None 4 ...
1,2001,"{'id': 1610612737, 'full_name': 'Atlanta Hawks...",0 52 1 50 2 49 3 44 4 44 5...,0 30 1 32 2 33 3 38 4 38 5...,0 0.634 1 0.610 2 0.598 3 0.53...,0 1 1 1 2 1 3 0 4 1 5 ...
2,2002,"{'id': 1610612737, 'full_name': 'Atlanta Hawks...",0 50 1 49 2 48 3 48 4 47 5...,0 32 1 33 2 34 3 34 4 35 5...,0 0.610 1 0.598 2 0.585 3 0.58...,0 1 1 1 2 1 3 1 4 1 5 ...
3,2003,"{'id': 1610612737, 'full_name': 'Atlanta Hawks...",0 61 1 47 2 54 3 42 4 41 5...,0 21 1 35 2 28 3 40 4 41 5...,0 0.744 1 0.573 2 0.659 3 0.51...,0 1 1 1 2 1 3 1 4 1 5 ...
4,2004,"{'id': 1610612737, 'full_name': 'Atlanta Hawks...",0 59 1 54 2 47 3 45 4 45 5...,0 23 1 28 2 35 3 37 4 37 5...,0 0.720 1 0.659 2 0.573 3 0.54...,0 1 1 1 2 1 3 1 4 1 5 ...
...,...,...,...,...,...,...
715,2019,"{'id': 1610612766, 'full_name': 'Charlotte Hor...",0 56 1 53 2 48 3 45 4 44 5...,0 17 1 19 2 24 3 28 4 29 5...,0 0.767 1 0.736 2 0.667 3 0.61...,0 1 1 1 2 1 3 1 4 1 5 ...
716,2020,"{'id': 1610612766, 'full_name': 'Charlotte Hor...",0 49 1 48 2 46 3 41 4 41 5...,0 23 1 24 2 26 3 31 4 31 5...,0 0.681 1 0.667 2 0.639 3 0.56...,0 1 1 1 2 1 3 1 4 1 5 ...
717,2021,"{'id': 1610612766, 'full_name': 'Charlotte Hor...",0 53 1 51 2 51 3 51 4 48 5...,0 29 1 31 2 31 3 31 4 34 5...,0 0.646 1 0.622 2 0.622 3 0.62...,0 1 1 1 2 1 3 1 4 1 5 ...
718,2022,"{'id': 1610612766, 'full_name': 'Charlotte Hor...",0 58 1 57 2 54 3 51 4 47 5...,0 24 1 25 2 28 3 31 4 35 5...,0 0.707 1 0.695 2 0.659 3 0.62...,0 1 1 1 2 1 3 1 4 1 5 ...
